In [34]:
from wikidataintegrator import wdi_core, wdi_login, wdi_helpers
from wikidataintegrator.wdi_helpers import try_write
from getpass import getpass
import ipywidgets as widgets
from IPython.display import IFrame, clear_output

## Login panel
### Label
wikidata = widgets.Label(value="Login to Wikidata")

### Username
username_textbox = widgets.Text(
       value="",
        description="Username",
)
## Password
password_textbox = widgets.Password(description='Password:')
## Button
loginbutton = widgets.Button(description="login")
## Login as
loginlabel = widgets.Label(value="")

@loginbutton.on_click
def wdlogin(b):
    login = wdi_login.WDLogin(username_textbox.value, password_textbox.value)
    creds = login.generate_edit_credentials()
    koekjes = creds.get_dict()
    loginlabel.value="Logged in to Wikidata as User: "+koekjes["wikidatawikiUserName"]
    b.login = login

loginPane = widgets.VBox(children=[wikidata, username_textbox, password_textbox, loginbutton, loginlabel])

### Tabs to identifier resolvers
#### Citations
doi_textbox = widgets.Text(
    value='',
    description='DOI',
)
     
wddoilabel = widgets.Label(value="")

doi_textbox = widgets.Text(
    value='',
    description='DOI',
)

progressbar = widgets.Output()


file = open("WikiciteWikidata.gif", "rb")
image = file.read()
progressgif = widgets.Image(
    value=image,
    format='gif',
    width=300,
    height=400,
)

# progressgif = IFrame(src="WikiciteWikidata", width="50", height="50")

DOIbutton = widgets.Button(
    description='Wikidata',
)
@DOIbutton.on_click
def DOIresolve(b):
    print(doi_textbox.value)
    with progressbar:
        clear_output()
        display(progressgif)
    doi_qid, _, _ = wdi_helpers.PublicationHelper(doi_textbox.value, id_type="doi",source="crossref").get_or_create(loginbutton.login)
    print(loginbutton.login)
    with progressbar:
        clear_output()

        ## Result tabs
        wikidatatab = IFrame(src='https://www.wikidata.org/wiki/'+doi_qid, width=1000, height=600)
        scholiatab = IFrame(src='https://scholia.toolforge.org/work/'+doi_qid, width=1000, height=600)
        tab1 = widgets.Output()
        tab2 = widgets.Output()
        tab = widgets.Tab(children=[
            tab1,
            tab2])
        with tab1:
            display(wikidatatab)
        with tab2:
            display(scholiatab)
        tab.set_title(0, 'Wikidata')
        tab.set_title(1, 'Scholia')
        display(tab)

form = widgets.VBox(children=[doi_textbox, DOIbutton, progressbar]) # Citations
widgets.VBox(children=[loginPane, form])